## step 1. 데이터 다운로드

## step 2. 데이터 읽어오기

In [1]:
import glob
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## step 3. 데이터 정제하기

In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


In [3]:
# 정제된 문장 모을 곳
corpus = []

for sentence in raw_corpus:
#     # 우리가 원하지 않는 문장은 건너뛴다
    if len(sentence) == 0: continue # 길이가 0인 문장 건너뛰기
    if sentence[-1] == ':': continue
    tmp = preprocess_sentence(sentence)
    if len(tmp.split()) > 15: continue
    corpus.append(tmp)
#     if len(sentence) == 0: continue

In [4]:
# 토큰화 => tensor로 만드는 과정
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 12000단어 기억가능한 tokenizer
        filters=' ', #정제를 했으므로 필터 필요x
        oov_token="<unk>" # 12000개에 포함 안되면 unk로
    )
    tokenizer.fit_on_texts(corpus) 
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용함
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc189020ee0>


In [5]:
# 텐서를 소스와 타겟으로 분리
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:] 

## step 4. 평가 데이터셋 분리

In [6]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=17)

In [7]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [8]:
# 데이터셋 객체 생성
BUFFER_SIZE = len(dec_train)
BATCH_SIZE = 256
steps_per_epoch = len(dec_train) // BATCH_SIZE
print(steps_per_epoch)

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듦
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

487


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## step 5. 인공지능 만들기

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)



In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data=(enc_val, dec_val),verbose=1)

Epoch 1/10
488/488 [==============================] - 268s 507ms/step - loss: 3.2773 - val_loss: 2.9160
Epoch 2/10
488/488 [==============================] - 254s 520ms/step - loss: 2.7283 - val_loss: 2.6559
Epoch 3/10
488/488 [==============================] - 254s 520ms/step - loss: 2.4088 - val_loss: 2.4782
Epoch 4/10
488/488 [==============================] - 254s 521ms/step - loss: 2.0994 - val_loss: 2.3450
Epoch 5/10
488/488 [==============================] - 255s 522ms/step - loss: 1.8086 - val_loss: 2.2499
Epoch 6/10
488/488 [==============================] - 254s 521ms/step - loss: 1.5520 - val_loss: 2.1857
Epoch 7/10
488/488 [==============================] - 255s 522ms/step - loss: 1.3439 - val_loss: 2.1562
Epoch 8/10
488/488 [==============================] - 254s 521ms/step - loss: 1.1858 - val_loss: 2.1587
Epoch 9/10
488/488 [==============================] - 255s 522ms/step - loss: 1.0817 - val_loss: 2.1719
Epoch 10/10
488/488 [==============================] - 255s 523m

In [11]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i ", max_len=20)

'<start> i m the only thing that you need <end> '

## 회고

### 어려웠던 점

- 데이터를 전처리하는 과정에서 '토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기'항목을 구현하는 것이 어려웠다.
- val_loss가 loss와의 차이를 인지하지 못해서 처음에 loss만 구했다.
- val_loss를 코드화하는 것이 어려웠다.

### 해결방안

- 부경님의 도움으로 줄이기는 했으나 노드에서 진행하라고 한 숫자와 차이가 있어서 계속 고민을 했다. 하지만 나중에 노드상에서 숫자가 오류가 있는 부분이 있어서 그 문제는 해결됐고 동시에 아지트에 올라온 지훈님의 글을 통해 조금 더 이해가 잘 되었다. (참조를 하려고했는데 회고를 적을 때 보니 와글와글에 글이 안보입니다ㅠㅠ)

- https://ebbnflow.tistory.com/122 를 참조하여 다시 보니 분명 배웠던 내용이었다...ㅠㅠ다시 한번 인지하고 넘어가는 것에 의의를 두었다.

- 코딩적인 부분은 https://velog.io/@hwanython/%EC%9E%91%EC%82%AC%EA%B0%80-%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5-%EB%A7%8C%EB%93%A4%EA%B8%B0 다음 블로그를 참조하여 해결했습니다.

### 느낀점 
사실 인공지능에 대해 분야도 제대로 모르는 상태에서 공부중이었다. 저번에는 CV이번에는 NLP분야를 배우게 되었는데 분야를 알아가는 재미도 있었고 각자가 다른 매력이 있었다. 이번 NLP에서의 어려움은 데이터를 정제해야하는 과정에서 사진과는 다르게 데이터가 가시적으로 보이는 것이 아니니 어떤 형태로 정제를 해야하는지가 감이 안오는 부분이 컸다. 매번 모델을 만들면서 느끼는 것인데 데이터 전처리과정이 가장 어려워보이는데 매번 어떤식으로 접근하는지 알아가기가 쉽지 않은 것 같다. 그래도 많은 경험을 하다 보면 내가 어떻게 해야할지 느끼는 바가 많아지지 않을까 하는 기대감을 갖고 이번 Exploration4 를 마치려고 한다.